In [11]:
import hashlib
import datetime
from collections import defaultdict

# Block Class
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash, nonce, miner):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash
        self.nonce = nonce
        self.miner = miner

    def __repr__(self):
        return (f"Block(Index: {self.index}, Hash: {self.hash}, Nonce: {self.nonce}, "
                f"Data: {self.data}, Miner: {self.miner})")

# Blockchain Class
class Blockchain:
    def __init__(self, difficulty=1, reward=50, miners=None):
        self.chain = []
        self.difficulty = difficulty
        self.reward = reward
        self.pending_rewards = defaultdict(int)
        self.miners = miners if miners else ["Miner 1", "Miner 2", "Miner 3"]

    def create_genesis_block(self):
        """Creates the first block in the blockchain."""
        genesis_block = Block(
            index=0,
            previous_hash="0",
            timestamp=datetime.datetime.now(),
            data="Genesis Block",
            hash="0",
            nonce=0,
            miner="System"
        )
        self.chain.append(genesis_block)
        return genesis_block

    def calculate_target(self):
        """Calculates the mining target based on difficulty."""
        return "0" * self.difficulty + "f" * (64 - self.difficulty)

    def proof_of_work(self, data, miner):
        """Performs mining using Proof of Work."""
        target = self.calculate_target()
        last_block = self.chain[-1]
        previous_hash = last_block.hash
        index = len(self.chain)
        timestamp = datetime.datetime.now()
        nonce = 0

        while True:
            hash_input = f"{index}{previous_hash}{timestamp}{data}{nonce}{miner}".encode()
            block_hash = hashlib.sha256(hash_input).hexdigest()

            if block_hash < target:
                # Successfully mined the block
                return Block(index, previous_hash, timestamp, data, block_hash, nonce, miner)

            nonce += 3  # Stricter nonce requirement (increment by 3)

    def consensus(self, block):
        """Simulates miner consensus to approve or reject a block."""
        votes = {miner: True if miner in self.miners else False for miner in self.miners}
        approval_count = sum(votes.values())
        return approval_count > len(self.miners) / 2  # Majority rule

    def mine_block(self, data, miner):
        """Attempts to mine a block and achieve consensus."""
        if miner not in self.miners:
            print(f"{miner} is not a registered miner.")
            return

        print(f"\n{miner} is mining...")
        mined_block = self.proof_of_work(data, miner)

        # Check consensus
        if self.consensus(mined_block):
            self.chain.append(mined_block)
            self.pending_rewards[miner] += self.reward
            print(f"Block added to chain by {miner} after consensus.")
        else:
            print(f"Block rejected: Consensus not reached.")

    def print_chain(self):
        """Prints all the blocks in the blockchain."""
        print("\nBlockchain:")
        for block in self.chain:
            print(block)

    def is_chain_valid(self):
        """Checks if the blockchain is valid."""
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            # Recalculate the hash
            hash_input = f"{current_block.index}{current_block.previous_hash}{current_block.timestamp}{current_block.data}{current_block.nonce}{current_block.miner}".encode()
            recalculated_hash = hashlib.sha256(hash_input).hexdigest()

            if current_block.hash != recalculated_hash:
                return False
            if current_block.previous_hash != previous_block.hash:
                return False

        return True

    def get_rewards(self, miner):
        """Gets the reward for a specific miner."""
        return self.pending_rewards[miner]

# Testing the Updated Blockchain
blockchain = Blockchain(difficulty=2, miners=["Miner 1", "Miner 2", "Miner 3"])

# Step 1: Create Genesis Block
print("Creating Genesis Block...")
blockchain.create_genesis_block()

# Step 2: Mine Blocks with Consensus
blockchain.mine_block("Transaction 1", "Miner 1")
blockchain.mine_block("Transaction 2", "Miner 2")
blockchain.mine_block("Transaction 3", "Miner 3")

# Step 3: Print the Blockchain
blockchain.print_chain()

# Step 4: Check Rewards
print("\nMining Rewards:")
for miner in blockchain.miners:
    print(f"{miner}: {blockchain.get_rewards(miner)} coins")

# Step 5: Validate Blockchain
print(f"\nBlockchain validity: {blockchain.is_chain_valid()}")

# Step 6: Tamper and Revalidate
# print("\nTampering with the Blockchain...")
# blockchain.chain[1].data = "Tampered Transaction"
# blockchain.print_chain()
# print(f"\nBlockchain validity after tampering: {blockchain.is_chain_valid()}")


Creating Genesis Block...

Miner 1 is mining...
Block added to chain by Miner 1 after consensus.

Miner 2 is mining...
Block added to chain by Miner 2 after consensus.

Miner 3 is mining...
Block added to chain by Miner 3 after consensus.

Blockchain:
Block(Index: 0, Hash: 0, Nonce: 0, Data: Genesis Block, Miner: System)
Block(Index: 1, Hash: 0068fda29f630033608b411d8f218ddee2b4e941663c320c60b6005538c54666, Nonce: 783, Data: Transaction 1, Miner: Miner 1)
Block(Index: 2, Hash: 0054367ff51bad2380f0ba9284d22838b4c465a489399c8765a85cfc2889950b, Nonce: 237, Data: Transaction 2, Miner: Miner 2)
Block(Index: 3, Hash: 000570698c6840ef5f456998c3bd1f8924d31fd7dfa54791cb689beee332f446, Nonce: 534, Data: Transaction 3, Miner: Miner 3)

Mining Rewards:
Miner 1: 50 coins
Miner 2: 50 coins
Miner 3: 50 coins

Blockchain validity: True
